<div class="contentcontainer med left" style="margin-left: -50px;">
<dl class="dl-horizontal">
  <dt>Description</dt> <dd> Streams quickstart guide</dd>
  <dt>Author</dt> <dd>Jean-Luc Stevens</dd>
  <dt>HoloViews</dt> <dd>>1.6.2</dd>
  <dt>Python</dt> <dd>2.7/3.3+</dd>
</dl>
</div>

This quickstart is designed to introduce the basics of the streams system planned for HoloViews 1.7. A stream is a simple object with a set of parameters that allow events to affect a displayable HoloViews object. Starting with a simple function to generate NumPy arrays, this quickstart guide serves as a short introduction to the ``Image`` element, ``DynamicMap`` and finally an example of how streams are used. This notebook uses core HoloViews together with the matplotlib backend.

In [ ]:
import holoviews as hv
import numpy as np
from holoviews.streams import Stream
hv.notebook_extension()

# Simple NumPy arrays

The following function generates 2D NumPy arrays that generates a pattern:

In [ ]:
xvals = np.linspace(-4,0,202)
yvals = np.linspace(4,0,202)
xs,ys = np.meshgrid(xvals, yvals)

def waves_array(alpha, beta):
    return np.sin(((ys/alpha)**alpha+beta)*xs)

Here is an example array generated using ``alpha=0`` and ``beta=1``. We can see it is a ``numpy.ndarray`` with a shape of ``(101,101)``.

In [ ]:
arr = waves_array(0,1)
print(type(arr))
print(arr.shape)

# ``hv.Image``

The following function generates an array with ``waves_array`` and passes is to a ``hv.Image`` element:

In [ ]:
def waves_image(alpha, beta):
    return hv.Image(waves_array(alpha, beta))

Here is an example of the output:

In [ ]:
waves_image(0,1)

And as with all HoloViews elements, you can use the ``+`` operator to generate a layout:

In [ ]:
waves_image(0,1) + waves_image(0.5,2)

For more information about HoloViews elements such as ``Image``, see the [Elements tutorial](http://holoviews.org/Tutorials/Elements.html). For more information about the ``+`` operator and ``Layout`` objects, see the [Containers tutorial](http://holoviews.org/Tutorials/Containers.html).

# DynamicMap

We can use our ``waves_image`` function in a HoloViews ``DynamicMap`` to interactively explore this alpha and beta space by declaring ``alpha`` and ``beta`` dimensions with appropriate ranges. Note that this example and those that follow need a live notebook kernel to work such as [this one on mybinder](http://mybinder.org/repo/ioam/holoviews-contrib/notebooks/notebooks/quickstart/Streams.ipynb):

In [ ]:
hv.DynamicMap(waves_image, kdims=[hv.Dimension('alpha',range=(0, np.pi)),
                                 hv.Dimension('beta', range=(0.01,np.pi))])

A DynamicMap takes a callable (such as a function) that returns HoloViews elements and uses it to dynamically update a visualization. In the example above, this callable receives the ``alpha`` and ``beta`` values as positional arguments as specified by the two numeric sliders. For more information about ``DynamicMap``, see the [DynamicMap tutorial](http://holoviews.org/Tutorials/Dynamic_Map.html).

## Streams

Streams are a new type of object introduced after the HoloViews 1.6.2 release. Many streams are ready to import and use directly but in this quickstart we will create a custom stream by subclassing from ``Stream``:

In [ ]:
import param
from holoviews.streams import Stream

class WaveParams(Stream):
    
    alpha = param.Number(default=0)
    
    beta = param.Number(default=0.01)

All that is necessary is to declare parameters using the [param library](http://ioam.github.io/param/). This is not a special property of Streams as almost all HoloViews objects are parametrized in this way. 

We can how make instances of this stream type:

In [ ]:
waves_params = WaveParams() 

print('Initial alpha: %s' % waves_params.alpha)
print('Initial beta: %s' % waves_params.beta)

As you can see, this instance uses the parameter defaults defined in the class definition above. We can also supply the values we want to initialize the stream with:

In [ ]:
waves_params = WaveParams(alpha=0.5, beta=0.1)

print('Initial alpha: %s' % waves_params.alpha)
print('Initial beta: %s' % waves_params.beta)

What streams do is to supply an ``update`` method to update these parameter values which will also update the visualization in the notebook as we will see in the next section:

In [ ]:
waves_params.update(alpha=0.3, beta=1)

print('Updated alpha: %s' % waves_params.alpha)
print('Updated beta: %s' % waves_params.beta)

The ``update`` method updates stream parameters and does nothing else. The ``event`` method on both ``Stream`` objects and ``DynamicMap`` behave just like ``update`` but also trigger the associated visualization to refresh as we will now show.

# Streams and DynamicMap

To use streams, you supply them to a ``DynamicMap`` using the ``streams`` argument instead of the ``kdims`` which were used to generate the slider example above:

In [ ]:
dmap = hv.DynamicMap(waves_image, kdims=[], streams=[waves_params])
dmap

The stream now lets us control the alpha and beta of the visualization above instead of using sliders. Running the following cell will double the beta parameter of the wave pattern (just as we saw this ``update`` in the previous section) and will update the visualization shown above:

In [ ]:
waves_params.event(beta=2)

We did not need to change the ``waves_image`` callback - whereas the arguments were passed in by position when ``kdims`` were used (see the sliders example above) they are now passed by name via keyword arguments.

As a ``DynamicMap`` can have access to the parameters to multiple streams, having to hold handles to the stream objects themselves can be awkward. The recommended way to update *any* of the available stream parameters is to call the ``event`` method on the ``DynamicMap`` itself:

In [ ]:
dmap.event(beta=3)

This approach only requires a handle on the DynamicMap (``dmap``), allowing you to declare the streams at the same time as you declare the rest of the ``DynamicMap``. In other words, we could have declared our dynamicmap in a single line using:

```
dmap = hv.DynamicMap(waves_image, kdims=[], streams=[WaveParams()])
```

Now we will update the alpha of the plot above using a simple loop:

In [ ]:
for alpha in np.linspace(0.1, 2):
    dmap.event(alpha=alpha)

Of course we can also update the beta and alpha values together:

In [ ]:
dmap.event(beta=2, alpha=1.5)

And then set it back to zero alpha:

In [ ]:
dmap.event(beta=2, alpha=0)

Now lets update the alpha in a loop - watch the plot animate above!

In [ ]:
alpha, beta = 0, 2
for i in range(600):
    alpha += 0.1
    dmap.event(alpha=alpha, beta=beta)

``DynamicMap`` objects with streams act like any other HoloViews objects - you can compose them in Layouts with ``+`` and use the option system too:

In [ ]:
%%opts Image (cmap='viridis')
layout = hv.DynamicMap(waves_image, kdims=[], streams=[WaveParams(beta=1)]) + waves_image(0,1)
layout

Now we can update the alpha in a loop by accessing the ``DynamicMap`` in the ``Layout`` and calling the ``event`` method:

In [ ]:
alpha = 0
for i in range(50):
    alpha += 0.3
    layout.DynamicMap.I.event(alpha=alpha)

Note that had we shared the stream instance (i.e used ``waves_params`` above) we could have used this loop to update two visualizations at once.

You will notice that the visualizations updates slower as the layout a more complex plot. For many elements, you will get get better performance using the [Bokeh backend](http://holoviews.org/Tutorials/Bokeh_Elements.html) and for more information about the options system, see the [Options tutorial](http://holoviews.org/Tutorials/Options.html).